<a href="https://colab.research.google.com/github/Riasci/KomputasiIntelegensiaTasks/blob/main/EndTermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Ria Mulyadi

NPM: 2206048556

Dataset: https://www.kaggle.com/datasets/zalando-research/fashionmnist

Model: CNN

Referensi Model: https://dlvr.rantai.dev/docs/part-ii/chapter-5/

In [ ]:
pip install torch requests aiohttp python-zlib anyio

ERROR: Could not find a version that satisfies the requirement python-zlib (from versions: none)
ERROR: No matching distribution found for python-zlib


In [ ]:
pip install torch torchvision requests

In [ ]:
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST
import os
from tqdm import tqdm

# Download Fashion MNIST dataset using kagglehub
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

# Define the path to the dataset
data_path = os.path.join(path, "fashionmnist", "versions", "4")  # Adjusted path

# Transformations for dataset (convert to tensor and normalize)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the Fashion MNIST dataset
train_dataset = FashionMNIST(data_path, train=True, download=True, transform=transform)
test_dataset = FashionMNIST(data_path, train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  # Reshape input to 1x28x28 images
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 1024)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

# Initialize the model and optimizer
model = Net().cuda()  # Move model to GPU if available
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training function
def train():
    model.train()
    for epoch in range(10):  # Training for 10 epochs
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.cuda(), labels.cuda()  # Move data to GPU
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update the weights
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# Testing function
def test():
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.cuda(), labels.cuda()  # Move data to GPU
            outputs = model(images)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

# Run the training and testing
train()
test()


Path to dataset files: /root/.cache/kagglehub/datasets/zalando-research/fashionmnist/versions/4


100%|██████████| 938/938 [00:16<00:00, 56.91it/s]


Epoch 1, Loss: 0.4687754220879281


100%|██████████| 938/938 [00:15<00:00, 59.61it/s]


Epoch 2, Loss: 0.30755398250909755


...

Epoch 9, Loss: 0.14002912203425855


100%|██████████| 938/938 [00:15<00:00, 59.43it/s]


Epoch 10, Loss: 0.1276080100717686


100%|██████████| 157/157 [00:02<00:00, 56.02it/s]

Accuracy: 91.21%
